# Implementing Gaussian Elimination

In [2]:
import numpy as np

We start with an implementation of Gaussian elimination as described previously.  Note that  a feature of this implementation is that the input `A` and `b` are changed by this routine, and on output they reflect the row-echelon
form.  This is done to save memory.

We also, by default, will print the steps along the way

In [4]:
def gauss_elim(A, b, quiet=0):
    """ perform gaussian elimination with pivoting, solving A x = b.

        A is an NxN matrix, x and b are an N-element vectors.  Note: A
        and b are changed upon exit to be in upper triangular (row
        echelon) form """

    assert b.ndim == 1, "ERROR: b should be a vector"

    N = len(b)

    # A is square, with each dimension of length N
    assert A.shape == (N, N), "ERROR: A should be square with each dim of same length as b"

    # allocation the solution array
    x = np.zeros((N), dtype=A.dtype)

    # find the scale factors for each row -- this is used when pivoting
    scales = np.max(np.abs(A), 1)

    # keep track of the number of times we swapped rows
    num_row_swap = 0

    if not quiet:
        print_Ab(A, b)

    # main loop over rows
    for k in range(N):

        # find the pivot row based on the size of column k -- only consider
        # the rows beyond the current row
        row_max = np.argmax(A[k:, k]/scales[k:])
        if k > 0:
            row_max += k  # we sliced A from k:, correct for total rows

        # swap the row with the largest scaled element in the current column
        # with the current row (pivot) -- do this with b too!
        if not row_max == k:
            A[[k, row_max], :] = A[[row_max, k], :]
            b[[k, row_max]] = b[[row_max, k]]
            scales[[k, row_max]] = scales[[row_max, k]]
            if not quiet:
                print("pivoted")
            num_row_swap += 1

        # do the forward-elimination for all rows below the current
        for i in range(k+1, N):
            coeff = A[i, k] / A[k, k]

            for j in range(k+1, N):
                A[i, j] += -A[k, j] * coeff

            A[i, k] = 0.0
            b[i] += -coeff * b[k]

        if not quiet:
            print_Ab(A, b)

    # back-substitution

    # last solution is easy
    x[N-1] = b[N-1] / A[N-1, N-1]

    for i in reversed(range(N-1)):
        bsum = b[i]
        for j in range(i+1, N):
            bsum += -A[i, j] * x[j]
        x[i] = bsum / A[i, i]

    return x

This routine prints the augmented matrix $({\bf A} | {\bf b})$.

In [5]:
def print_Ab(A, b):
    """ printout the matrix A and vector b in a pretty fashion.  We
        don't use the numpy print here, because we want to make them
        side by side"""

    N = len(b)

    # draw pretty brackets

    topl = "\u256d"
    topr = "\u256e"
    side = "\u2502"
    botl = "\u2570"
    botr = "\u256F"

    space = 8*" "

    top_str = topl + N*" {:>7.03f} " + topr + space + topl + " {:6.3f} " + topr
    sid_str = side + N*" {:>7.03f} " + side + space + side + " {:6.3f} " + side
    bot_str = botl + N*" {:>7.03f} " + botr + space + botl + " {:6.3f} " + botr

    print(" ")
    for i in range(N):
        if i == 0:
            pstr = top_str
        elif i == N-1:
            pstr = bot_str
        else:
            pstr = sid_str
        out = tuple(A[i, :]) + (b[i],)
        print(pstr.format(*out))

Now we can test this out.

In [8]:
A = np.array([[1, 1, 1],
              [-1, 2, 0],
              [2, 0, 1]], dtype=np.float64)
b = np.array([6, 3, 5], dtype=np.float64)
x = gauss_elim(A, b)

 
╭   1.000    1.000    1.000 ╮        ╭  6.000 ╮
│  -1.000    2.000    0.000 │        │  3.000 │
╰   2.000    0.000    1.000 ╯        ╰  5.000 ╯
 
╭   1.000    1.000    1.000 ╮        ╭  6.000 ╮
│   0.000    3.000    1.000 │        │  9.000 │
╰   0.000   -2.000   -1.000 ╯        ╰ -7.000 ╯
 
╭   1.000    1.000    1.000 ╮        ╭  6.000 ╮
│   0.000    3.000    1.000 │        │  9.000 │
╰   0.000    0.000   -0.333 ╯        ╰ -1.000 ╯
 
╭   1.000    1.000    1.000 ╮        ╭  6.000 ╮
│   0.000    3.000    1.000 │        │  9.000 │
╰   0.000    0.000   -0.333 ╯        ╰ -1.000 ╯


In [9]:
print(x)

[1. 2. 3.]


To ensure that we got the right answer, we can compare ${\bf A}{\bf x}$ and ${\bf b}$.  We can use the python matrix multiplication operator, `@`:

In [11]:
print(A @ x - b)

[0. 0. 0.]
